In [23]:
# 将MONTH列拆分为年份和月份两列

import pandas as pd

# 读取训练集和测试集
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# 从MONTH列中提取年份和月份
# MONTH格式为 "Oct-20" 或 "Jul-21"，需要解析月份缩写和2位年份
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

def extract_year_month(month_str):
    """
    从 MONTH 列中提取年份和月份
    支持两种格式：
    1. 'Oct-20' 或 'Jul-21'（月份缩写-2位年份）
    2. '2025-06' 或 '2020-01'（YYYY-MM格式）
    """
    parts = month_str.split('-')
    
    # 检查是否为 YYYY-MM 格式（第一部分是4位数字）
    if len(parts[0]) == 4 and parts[0].isdigit():
        # 格式：2025-06
        year = int(parts[0])
        month = int(parts[1])
    else:
        # 格式：Oct-20 或 Jul-21
        month_abbr = parts[0]
        year_2digit = int(parts[1])
        
        # 将2位年份转换为4位年份（假设20-99表示2000-2099）
        year = 2000 + year_2digit if year_2digit < 100 else year_2digit
        month = month_map.get(month_abbr, 0)
    
    return year, month

# 应用函数提取年份和月份
df_train[['YEAR', 'MONTH_INT']] = df_train['MONTH'].apply(
    lambda x: pd.Series(extract_year_month(x))
)
df_test[['YEAR', 'MONTH_INT']] = df_test['MONTH'].apply(
    lambda x: pd.Series(extract_year_month(x))
)

# 保存修改后的文件（覆盖原文件）
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

print("文件已更新：train.csv 和 test.csv")
print(f"训练集形状: {df_train.shape}")
print(f"测试集形状: {df_test.shape}")
print("\n训练集前5行:")
print(df_train[["MONTH", "YEAR", "MONTH_INT"]].head())
print("\n测试集前5行:")
print(df_test[["MONTH", "YEAR", "MONTH_INT"]].head())


文件已更新：train.csv 和 test.csv
训练集形状: (162691, 15)
测试集形状: (50000, 20)

训练集前5行:
    MONTH  YEAR  MONTH_INT
0  Oct-20  2020         10
1  Jul-21  2021          7
2  May-21  2021          5
3  Aug-21  2021          8
4  May-23  2023          5

测试集前5行:
     MONTH  YEAR  MONTH_INT
0  2025-06  2025          6
1  2020-01  2020          1
2  2025-06  2025          6
3  2022-10  2022         10
4  2024-02  2024          2


In [24]:
# 处理FLAT_TYPE列：拆分为IS_EXECUTIVE和NUM_ROOMS两列

import re

def process_flat_type(flat_type_str):
    """
    处理FLAT_TYPE列：
    - IS_EXECUTIVE: 如果是executive则为1，否则为0
    - NUM_ROOMS: 如果是executive则为0，否则从"N-room"或"N room"格式中提取N
    """
    flat_type_str = str(flat_type_str).strip().lower()
    
    # 检查是否为executive
    if flat_type_str == 'executive':
        return 1, 0
    
    # 尝试从"N-room"或"N room"格式中提取房间数
    # 匹配模式：数字后跟"-room"或" room"
    match = re.search(r'(\d+)[\s-]?room', flat_type_str)
    if match:
        num_rooms = int(match.group(1))
        return 0, num_rooms
    
    # 如果无法匹配（如"multi generation"），返回0, 0
    return 0, 0

# 应用函数处理FLAT_TYPE列
df_train[['IS_EXECUTIVE', 'NUM_ROOMS']] = df_train['FLAT_TYPE'].apply(
    lambda x: pd.Series(process_flat_type(x))
)
df_test[['IS_EXECUTIVE', 'NUM_ROOMS']] = df_test['FLAT_TYPE'].apply(
    lambda x: pd.Series(process_flat_type(x))
)

# 保存修改后的文件（覆盖原文件）
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

print("文件已更新：train.csv 和 test.csv")
print(f"训练集形状: {df_train.shape}")
print(f"测试集形状: {df_test.shape}")
print("\nFLAT_TYPE处理结果统计:")
print("训练集:")
print(df_train[['FLAT_TYPE', 'IS_EXECUTIVE', 'NUM_ROOMS']].value_counts().head(10))
print("\n测试集:")
print(df_test[['FLAT_TYPE', 'IS_EXECUTIVE', 'NUM_ROOMS']].value_counts().head(10))
print("\n训练集前10行:")
print(df_train[['FLAT_TYPE', 'IS_EXECUTIVE', 'NUM_ROOMS']].head(10))
print("\n测试集前10行:")
print(df_test[['FLAT_TYPE', 'IS_EXECUTIVE', 'NUM_ROOMS']].head(10))


文件已更新：train.csv 和 test.csv
训练集形状: (162691, 17)
测试集形状: (50000, 22)

FLAT_TYPE处理结果统计:
训练集:
FLAT_TYPE         IS_EXECUTIVE  NUM_ROOMS
4 room            0             4            48459
5 room            0             5            27916
3 room            0             3            26702
4-room            0             4            20414
5-room            0             5            12167
3-room            0             3            11904
executive         1             0            11801
2 room            0             2             2503
2-room            0             2              700
multi generation  0             0               68
Name: count, dtype: int64

测试集:
FLAT_TYPE         IS_EXECUTIVE  NUM_ROOMS
4 room            0             4            14939
5 room            0             5             8483
3 room            0             3             8307
4-room            0             4             6187
5-room            0             5             3749
3-room            0           

In [25]:
# 处理FLOOR_RANGE列：将楼层范围转换为有序编码

import re
import numpy as np
from bisect import bisect_left

# 正则表达式匹配 "数字 to 数字" 格式
_rng = re.compile(r'(\d+)\s*to\s*(\d+)', flags=re.IGNORECASE)

def parse_floor_range(x):
    """解析楼层范围字符串，返回 (lo, hi) 元组"""
    if pd.isna(x):
        return None
    s = str(x).strip()
    m = _rng.search(s)
    if not m:
        return None
    lo, hi = int(m.group(1)), int(m.group(2))
    if lo > hi:
        lo, hi = hi, lo
    return lo, hi

def build_floor_order_mapping(df_train, col="FLOOR_RANGE"):
    """
    基于训练集构建楼层范围的排序映射
    返回: mapping字典, pairs列表, lows列表, median_code
    """
    pairs = (
        df_train[col]
        .dropna()
        .map(parse_floor_range)
        .dropna()
        .unique()
    )
    pairs = sorted(pairs, key=lambda t: (t[0], t[1]))
    mapping = {f"{lo:02d} to {hi:02d}": i for i, (lo, hi) in enumerate(pairs)}
    for i, (lo, hi) in enumerate(pairs):
        mapping[f"{lo} to {hi}"] = i

    lows = [lo for (lo, _) in pairs]
    median_code = int(np.median(list(mapping.values()))) if mapping else 0
    return mapping, pairs, lows, median_code

def encode_floor_range_series(sr, mapping, pairs, lows, median_code):
    """
    将楼层范围序列编码为有序整数
    对于无法直接映射的值，使用fallback策略找到最接近的楼层范围
    """
    enc = sr.map(mapping)
    mask_na = enc.isna()
    if mask_na.any():
        parsed = sr[mask_na].map(parse_floor_range)

        def fallback_code(p):
            if p is None:
                return median_code
            lo, hi = p
            if not lows:
                return median_code
            pos = bisect_left(lows, lo)
            pos = max(0, min(pos, len(pairs)-1))
            return pos

        enc.loc[mask_na] = parsed.map(fallback_code)

    return enc.astype(int)

# 基于训练集构建映射
mapping, pairs, lows, median_code = build_floor_order_mapping(df_train, col="FLOOR_RANGE")

# 应用编码到训练集和测试集
df_train["FLOOR_RANGE_ORD"] = encode_floor_range_series(
    df_train["FLOOR_RANGE"], mapping, pairs, lows, median_code
)

df_test["FLOOR_RANGE_ORD"] = encode_floor_range_series(
    df_test["FLOOR_RANGE"], mapping, pairs, lows, median_code
)

# 保存修改后的文件（覆盖原文件）
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

print("文件已更新：train.csv 和 test.csv")
print(f"训练集形状: {df_train.shape}")
print(f"测试集形状: {df_test.shape}")
print("\nFLOOR_RANGE编码结果统计:")
print("训练集前10行:")
print(df_train[["FLOOR_RANGE", "FLOOR_RANGE_ORD"]].head(10))
print("\n测试集前10行:")
print(df_test[["FLOOR_RANGE", "FLOOR_RANGE_ORD"]].head(10))
print("\n编码值分布（训练集）:")
print(df_train["FLOOR_RANGE_ORD"].value_counts().sort_index().head(10))


文件已更新：train.csv 和 test.csv
训练集形状: (162691, 18)
测试集形状: (50000, 23)

FLOOR_RANGE编码结果统计:
训练集前10行:
  FLOOR_RANGE  FLOOR_RANGE_ORD
0    07 to 09                2
1    07 to 09                2
2    19 to 21                6
3    16 to 18                5
4    10 to 12                3
5    07 to 09                2
6    13 to 15                4
7    07 to 09                2
8    04 to 06                1
9    01 to 03                0

测试集前10行:
  FLOOR_RANGE  FLOOR_RANGE_ORD
0    04 to 06                1
1    01 to 03                0
2    10 to 12                3
3    10 to 12                3
4    10 to 12                3
5    04 to 06                1
6    10 to 12                3
7    01 to 03                0
8    13 to 15                4
9    04 to 06                1

编码值分布（训练集）:
FLOOR_RANGE_ORD
0    28682
1    37358
2    34103
3    30239
4    15812
5     7342
6     3147
7     2212
8     1392
9      887
Name: count, dtype: int64


In [26]:
# 处理FLAT_MODEL列：使用Target Encoding（K-Fold CV）

from sklearn.model_selection import KFold

def te_cv_train(df, col="FLAT_MODEL", target_col="RESALE_PRICE",
                n_splits=5, out_col=None, random_state=0):
    """
    K-Fold CV Target Encoding.
    使用交叉验证避免数据泄露。
    """
    df = df.copy()
    if out_col is None:
        out_col = f"{col}_TARGET_ENC"
    df[out_col] = pd.NA

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for tr_idx, va_idx in kf.split(df):
        tr = df.iloc[tr_idx]
        va = df.iloc[va_idx]

        # 计算训练集中每个类别的目标变量均值
        mean_map = tr.groupby(col)[target_col].mean().to_dict()
        s = df.loc[va.index, col].map(mean_map)
        # 对于未见过的类别，使用全局均值
        fold_global = tr[target_col].mean()
        s = s.fillna(fold_global)

        df.loc[va.index, out_col] = s

    df[out_col] = df[out_col].astype(float)
    # 填充任何剩余的缺失值（使用全局均值）
    df[out_col] = df[out_col].fillna(df[target_col].mean())

    return df

# 对训练集进行Target Encoding
df_train = te_cv_train(df_train, col="FLAT_MODEL", target_col="RESALE_PRICE", 
                       n_splits=5, out_col="FLAT_MODEL_TARGET_ENC")

# 对测试集：使用训练集的统计信息进行编码
# 计算训练集中每个FLAT_MODEL的RESALE_PRICE均值
mean_map = df_train.groupby('FLAT_MODEL')['RESALE_PRICE'].mean().to_dict()
global_mean = df_train['RESALE_PRICE'].mean()

# 对测试集应用编码
df_test['FLAT_MODEL_TARGET_ENC'] = df_test['FLAT_MODEL'].map(mean_map)
# 对于测试集中未见过的类别，使用全局均值
df_test['FLAT_MODEL_TARGET_ENC'] = df_test['FLAT_MODEL_TARGET_ENC'].fillna(global_mean)
df_test['FLAT_MODEL_TARGET_ENC'] = df_test['FLAT_MODEL_TARGET_ENC'].astype(float)

# 保存修改后的文件（覆盖原文件）
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

print("文件已更新：train.csv 和 test.csv")
print(f"训练集形状: {df_train.shape}")
print(f"测试集形状: {df_test.shape}")
print(f"\nFLAT_MODEL唯一值数量: {len(df_train['FLAT_MODEL'].unique())}")
print("\nFLAT_MODEL唯一值列表:")
for i, model in enumerate(sorted(df_train['FLAT_MODEL'].unique()), 1):
    print(f"{i:2d}. {model}")
print("\n训练集Target Encoding统计:")
print(df_train[["FLAT_MODEL", "FLAT_MODEL_TARGET_ENC"]].head(10))
print("\n各FLAT_MODEL的Target Encoding值（按编码值排序）:")
encoding_stats = df_train.groupby('FLAT_MODEL')['FLAT_MODEL_TARGET_ENC'].mean().sort_values()
print(encoding_stats)
print("\n测试集Target Encoding统计:")
print(df_test[["FLAT_MODEL", "FLAT_MODEL_TARGET_ENC"]].head(10))


文件已更新：train.csv 和 test.csv
训练集形状: (162691, 19)
测试集形状: (50000, 24)

FLAT_MODEL唯一值数量: 21

FLAT_MODEL唯一值列表:
 1. 2 room
 2. 3gen
 3. adjoined flat
 4. apartment
 5. dbss
 6. improved
 7. improved maisonette
 8. maisonette
 9. model a
10. model a maisonette
11. model a2
12. multi generation
13. new generation
14. premium apartment
15. premium apartment loft
16. premium maisonette
17. simplified
18. standard
19. terrace
20. type s1
21. type s2

训练集Target Encoding统计:
          FLAT_MODEL  FLAT_MODEL_TARGET_ENC
0  premium apartment          558599.442394
1            model a          510078.105938
2            model a          510293.613164
3            model a          510754.108089
4           improved          528016.555202
5     new generation          380425.778851
6            model a          510754.108089
7     new generation          379755.152358
8  premium apartment          558544.771495
9            model a          510078.105938

各FLAT_MODEL的Target Encoding值（按编码值排序）:
FLAT_MODEL
2

In [27]:
# 基于LEASE_COMMENCE_DATA创建房龄特征
# 房龄 = 售卖年份 - 地契开始年份

import pandas as pd

# 读取训练集和测试集
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# 计算房龄（从地契开始到售卖时经过的年数）
df_train["FLAT_AGE"] = df_train["YEAR"] - df_train["LEASE_COMMENCE_DATA"]
df_test["FLAT_AGE"] = df_test["YEAR"] - df_test["LEASE_COMMENCE_DATA"]

# 检查是否有异常值（房龄为负数或过大）
print("=== 房龄特征统计 ===")
print(f"训练集房龄范围: {df_train['FLAT_AGE'].min()} - {df_train['FLAT_AGE'].max()} 年")
print(f"测试集房龄范围: {df_test['FLAT_AGE'].min()} - {df_test['FLAT_AGE'].max()} 年")

# 检查异常值
train_negative = (df_train["FLAT_AGE"] < 0).sum()
test_negative = (df_test["FLAT_AGE"] < 0).sum()
train_too_old = (df_train["FLAT_AGE"] > 99).sum()  # 超过99年地契期限
test_too_old = (df_test["FLAT_AGE"] > 99).sum()

print(f"\n异常值检查:")
print(f"训练集中房龄为负数的样本数: {train_negative}")
print(f"测试集中房龄为负数的样本数: {test_negative}")
print(f"训练集中房龄超过99年的样本数: {train_too_old}")
print(f"测试集中房龄超过99年的样本数: {test_too_old}")

# 显示房龄分布统计
print(f"\n训练集房龄统计:")
print(df_train["FLAT_AGE"].describe())
print(f"\n测试集房龄统计:")
print(df_test["FLAT_AGE"].describe())

# 显示房龄分布（前10个最常见的值）
print(f"\n训练集房龄分布（前10个最常见的值）:")
print(df_train["FLAT_AGE"].value_counts().sort_index().head(10))

# 保存修改后的文件（覆盖原文件）
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

print("\n文件已更新：train.csv 和 test.csv")
print(f"训练集形状: {df_train.shape}")
print(f"测试集形状: {df_test.shape}")

# 显示示例数据
print("\n示例数据（前10行）:")
print(df_train[["YEAR", "LEASE_COMMENCE_DATA", "FLAT_AGE", "RESALE_PRICE"]].head(10))


=== 房龄特征统计 ===
训练集房龄范围: 1 - 59 年
测试集房龄范围: 1 - 58 年

异常值检查:
训练集中房龄为负数的样本数: 0
测试集中房龄为负数的样本数: 0
训练集中房龄超过99年的样本数: 0
测试集中房龄超过99年的样本数: 0

训练集房龄统计:
count    162691.000000
mean         24.691956
std          14.137947
min           1.000000
25%          10.000000
50%          25.000000
75%          36.000000
max          59.000000
Name: FLAT_AGE, dtype: float64

测试集房龄统计:
count    50000.000000
mean        24.650980
std         14.159893
min          1.000000
25%         10.000000
50%         25.000000
75%         36.000000
max         58.000000
Name: FLAT_AGE, dtype: float64

训练集房龄分布（前10个最常见的值）:
FLAT_AGE
1         1
2        41
3       811
4      9871
5     10545
6      6729
7      4684
8      3157
9      2790
10     2210
Name: count, dtype: int64

文件已更新：train.csv 和 test.csv
训练集形状: (162691, 20)
测试集形状: (50000, 25)

示例数据（前10行）:
   YEAR  LEASE_COMMENCE_DATA  FLAT_AGE  RESALE_PRICE
0  2020                 2000        20      420000.0
1  2021                 1992        29      585000.0
2  2021     

In [28]:
# 删除不需要的列：

import pandas as pd

# 读取训练集和测试集
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# 要删除的列
cols_to_drop = ['MONTH', 'FLAT_TYPE', 'FLOOR_RANGE', 'FLAT_MODEL', 'ECO_CATEGORY']

# 检查哪些列存在
train_cols_exist = [col for col in cols_to_drop if col in df_train.columns]
test_cols_exist = [col for col in cols_to_drop if col in df_test.columns]

print(f"训练集中存在的要删除的列: {train_cols_exist}")
print(f"测试集中存在的要删除的列: {test_cols_exist}")

# 删除列
if train_cols_exist:
    df_train = df_train.drop(columns=train_cols_exist)
    print(f"\n从训练集中删除了列: {train_cols_exist}")

if test_cols_exist:
    df_test = df_test.drop(columns=test_cols_exist)
    print(f"从测试集中删除了列: {test_cols_exist}")

# 保存修改后的文件（覆盖原文件）
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

print("\n文件已更新：train.csv 和 test.csv")
print(f"训练集新形状: {df_train.shape}")
print(f"测试集新形状: {df_test.shape}")
print(f"\n训练集剩余列名:")
print(df_train.columns.tolist())
print(f"\n测试集剩余列名:")
print(df_test.columns.tolist())


训练集中存在的要删除的列: ['MONTH', 'FLAT_TYPE', 'FLOOR_RANGE', 'FLAT_MODEL', 'ECO_CATEGORY']
测试集中存在的要删除的列: ['MONTH', 'FLAT_TYPE', 'FLOOR_RANGE', 'FLAT_MODEL', 'ECO_CATEGORY']

从训练集中删除了列: ['MONTH', 'FLAT_TYPE', 'FLOOR_RANGE', 'FLAT_MODEL', 'ECO_CATEGORY']
从测试集中删除了列: ['MONTH', 'FLAT_TYPE', 'FLOOR_RANGE', 'FLAT_MODEL', 'ECO_CATEGORY']

文件已更新：train.csv 和 test.csv
训练集新形状: (162691, 15)
测试集新形状: (50000, 20)

训练集剩余列名:
['TOWN', 'BLOCK', 'STREET', 'FLOOR_AREA_SQM', 'LEASE_COMMENCE_DATA', 'RESALE_PRICE', 'LATITUDE', 'LONGITUDE', 'YEAR', 'MONTH_INT', 'IS_EXECUTIVE', 'NUM_ROOMS', 'FLOOR_RANGE_ORD', 'FLAT_MODEL_TARGET_ENC', 'FLAT_AGE']

测试集剩余列名:
['TOWN', 'BLOCK', 'STREET', 'FLOOR_AREA_SQM', 'LEASE_COMMENCE_DATA', 'DIST_MRT_MIN', 'DIST_PRIMARY_SCHOOL_MIN', 'DIST_SECONDARY_SCHOOL_MIN', 'DIST_MALL_MIN', 'DIST_HAWKER_MIN', 'address', 'LATITUDE', 'LONGITUDE', 'YEAR', 'MONTH_INT', 'IS_EXECUTIVE', 'NUM_ROOMS', 'FLOOR_RANGE_ORD', 'FLAT_MODEL_TARGET_ENC', 'FLAT_AGE']


In [29]:
# 计算每个数据点到各类型设施最近的5个距离的标准化值作为特征
# 对于每种设施类型（hawker, station, primaryschool, secondaryschool, shoppingmall），
# 找到每个数据点最近的5个设施，计算距离并标准化

import pandas as pd
import numpy as np
import os
import torch

# 检查CUDA是否可用
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用设备: {device}")

def haversine_torch(lat1, lon1, lat2, lon2, device=device):
    """
    使用PyTorch计算Haversine距离（单位：公里）
    """
    lat1 = torch.deg2rad(torch.tensor(lat1, dtype=torch.float32, device=device))
    lon1 = torch.deg2rad(torch.tensor(lon1, dtype=torch.float32, device=device))
    lat2 = torch.deg2rad(torch.tensor(lat2, dtype=torch.float32, device=device))
    lon2 = torch.deg2rad(torch.tensor(lon2, dtype=torch.float32, device=device))
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = torch.sin(dlat/2)**2 + torch.cos(lat1) * torch.cos(lat2) * torch.sin(dlon/2)**2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1-a))
    
    R = 6371.0  # 地球半径（公里）
    return R * c

def calculate_nearest_k_distances(df_main, df_aux, k=5, lat_col='LATITUDE', lon_col='LONGITUDE', device=device):
    """
    计算主数据集中每个点到辅助数据集中最近的k个点的距离（完全向量化版本）
    返回: numpy数组，形状为 (len(df_main), k)，每行包含最近的k个距离（按距离从小到大排序）
    """
    # 提取主数据集的经纬度并转换为tensor
    main_lat = torch.tensor(df_main[lat_col].values, dtype=torch.float32, device=device)
    main_lon = torch.tensor(df_main[lon_col].values, dtype=torch.float32, device=device)
    
    # 提取辅助数据集的经纬度并转换为tensor
    aux_lat = torch.tensor(df_aux[lat_col].values, dtype=torch.float32, device=device)
    aux_lon = torch.tensor(df_aux[lon_col].values, dtype=torch.float32, device=device)
    
    # 创建掩码，标记有效的经纬度
    main_valid = ~(torch.isnan(main_lat) | torch.isnan(main_lon))
    aux_valid = ~(torch.isnan(aux_lat) | torch.isnan(aux_lon))
    
    # 过滤掉无效的辅助数据点
    aux_lat_valid = aux_lat[aux_valid]
    aux_lon_valid = aux_lon[aux_valid]
    
    if len(aux_lat_valid) == 0:
        return np.full((len(df_main), k), np.nan)
    
    # 转换为弧度
    main_lat_rad = torch.deg2rad(main_lat)
    main_lon_rad = torch.deg2rad(main_lon)
    aux_lat_rad = torch.deg2rad(aux_lat_valid)
    aux_lon_rad = torch.deg2rad(aux_lon_valid)
    
    # 使用广播计算所有距离矩阵
    # main_lat_rad: (N,), aux_lat_rad: (M,)
    # 扩展维度以进行广播: (N, 1) 和 (1, M) -> (N, M)
    main_lat_expanded = main_lat_rad.unsqueeze(1)  # (N, 1)
    main_lon_expanded = main_lon_rad.unsqueeze(1)  # (N, 1)
    aux_lat_expanded = aux_lat_rad.unsqueeze(0)    # (1, M)
    aux_lon_expanded = aux_lon_rad.unsqueeze(0)    # (1, M)
    
    # 计算Haversine距离（向量化）
    dlat = aux_lat_expanded - main_lat_expanded  # (N, M)
    dlon = aux_lon_expanded - main_lon_expanded  # (N, M)
    
    a = torch.sin(dlat/2)**2 + torch.cos(main_lat_expanded) * torch.cos(aux_lat_expanded) * torch.sin(dlon/2)**2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1-a))
    R = 6371.0  # 地球半径（公里）
    distances_matrix = R * c  # (N, M)
    
    # 将无效主数据点的距离设为inf，这样它们不会被选为最近的点
    distances_matrix[~main_valid, :] = float('inf')
    
    # 转换为numpy数组以便使用numpy的排序函数
    if device == 'cuda':
        distances_matrix = distances_matrix.cpu().numpy()
    else:
        distances_matrix = distances_matrix.numpy()
    
    # 找到每个主数据点最近的k个距离
    k_actual = min(k, distances_matrix.shape[1])
    if k_actual == 0:
        return np.full((len(df_main), k), np.nan)
    
    # 使用argpartition找到最小的k个元素（比完全排序更快）
    nearest_indices = np.argpartition(distances_matrix, k_actual - 1, axis=1)[:, :k_actual]
    
    # 提取对应的距离值并排序
    nearest_distances = np.full((len(df_main), k), np.nan)
    for i in range(len(df_main)):
        if not main_valid[i]:
            continue
        nearest_k = distances_matrix[i, nearest_indices[i]]
        nearest_k = np.sort(nearest_k)
        # 如果找到的距离少于k个，用最后一个距离填充
        if len(nearest_k) < k:
            nearest_k = np.pad(nearest_k, (0, k - len(nearest_k)), mode='edge')
        nearest_distances[i, :] = nearest_k[:k]
    
    return nearest_distances

# 读取训练集和测试集
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# 定义设施类型和对应的文件名映射
facility_mapping = {
    'hawker': 'sg-gov-hawkers.csv',
    'station': 'sg-mrt-stations.csv',
    'primaryschool': 'sg-primary-schools.csv',
    'secondaryschool': 'sg-secondary-schools.csv',
    'shoppingmall': 'sg-shopping-malls.csv'
}

auxiliary_dir = "auxiliary-data"
k_nearest = 5  # 最近的5个距离

print("="*60)
print(f"计算每个数据点到各类型设施最近的{k_nearest}个距离...")
print("="*60)

# 收集训练集距离用于计算标准化参数（避免数据泄露）
train_distances = []

# 为每种设施类型计算最近的k个距离
for facility_type, filename in facility_mapping.items():
    aux_file = os.path.join(auxiliary_dir, filename)
    
    if not os.path.exists(aux_file):
        print(f"\n警告: 文件 {filename} 不存在，跳过 {facility_type}")
        continue
    
    # 读取辅助数据
    df_aux = pd.read_csv(aux_file)
    
    # 检查是否有LATITUDE和LONGITUDE列
    if 'LATITUDE' not in df_aux.columns or 'LONGITUDE' not in df_aux.columns:
        print(f"\n警告: {filename} 缺少LATITUDE或LONGITUDE列，跳过 {facility_type}")
        continue
    
    # 过滤掉缺失经纬度的行
    df_aux = df_aux.dropna(subset=['LATITUDE', 'LONGITUDE']).reset_index(drop=True)
    
    if len(df_aux) == 0:
        print(f"\n警告: {filename} 没有有效的经纬度数据，跳过 {facility_type}")
        continue
    
    # 对重复的经纬度进行去重（同一个位置只保留一个）
    # 这对于MRT站很重要，因为同一个站可能有多条线路
    original_count = len(df_aux)
    df_aux = df_aux.drop_duplicates(subset=['LATITUDE', 'LONGITUDE']).reset_index(drop=True)
    unique_count = len(df_aux)
    
    print(f"\n处理 {facility_type} ({filename}):")
    print(f"  原始数据点数: {original_count}")
    print(f"  去重后数据点数: {unique_count} (去除了 {original_count - unique_count} 个重复位置)")
    
    # 计算训练集最近的k个距离
    print("  计算训练集最近的5个距离...")
    train_nearest = calculate_nearest_k_distances(df_train, df_aux, k=k_nearest, device=device)
    
    # 计算测试集最近的k个距离
    print("  计算测试集最近的5个距离...")
    test_nearest = calculate_nearest_k_distances(df_test, df_aux, k=k_nearest, device=device)
    
    # 只收集训练集距离用于计算标准化参数（避免数据泄露）
    train_valid = ~np.isnan(train_nearest)
    train_distances.extend(train_nearest[train_valid].tolist())
    
    # 创建特征列名并添加到DataFrame
    for i in range(k_nearest):
        col_name = f"DIST_NEAREST_{k_nearest}_{facility_type.upper()}_{i+1}"
        df_train[col_name] = train_nearest[:, i]
        df_test[col_name] = test_nearest[:, i]
    
    print(f"  已创建 {k_nearest} 个特征列")

# 统一标准化：只使用训练集距离计算min和max（避免数据泄露）
print("\n" + "="*60)
print("统一标准化所有最近距离特征（仅使用训练集计算标准化参数）...")
train_distances = np.array(train_distances)
dmin = np.min(train_distances)
dmax = np.max(train_distances)

print(f"训练集距离范围: [{dmin:.4f}, {dmax:.4f}] 公里")
print(f"训练集距离值数量: {len(train_distances)}")
print("注意: 使用训练集的min/max来标准化训练集和测试集，避免数据泄露")

# 对每个最近距离特征进行标准化
print("\n标准化所有最近距离特征...")
for facility_type in facility_mapping.keys():
    for i in range(k_nearest):
        col_name = f"DIST_NEAREST_{k_nearest}_{facility_type.upper()}_{i+1}"
        if col_name in df_train.columns:
            # 标准化公式: 1 - (d - dmin) / (dmax - dmin)
            if dmax > dmin:
                df_train[col_name] = (1 - (df_train[col_name] - dmin) / (dmax - dmin)).fillna(0.0)
                df_test[col_name] = (1 - (df_test[col_name] - dmin) / (dmax - dmin)).fillna(0.0)
            else:
                df_train[col_name] = df_train[col_name].fillna(1.0)
                df_test[col_name] = df_test[col_name].fillna(1.0)

# 保存修改后的文件
print("\n保存文件...")
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

print("\n" + "="*60)
print("文件已更新：train.csv 和 test.csv")
print(f"训练集形状: {df_train.shape}")
print(f"测试集形状: {df_test.shape}")

# 显示创建的特征统计
print(f"\n创建的最近距离特征数: {len(facility_mapping) * k_nearest} 个")
print("\n特征列名示例（前10个）:")
nearest_cols = [col for col in df_train.columns if col.startswith('DIST_NEAREST_')]
for i, col in enumerate(nearest_cols[:10], 1):
    if col in df_train.columns:
        print(f"  {i:2d}. {col}: min={df_train[col].min():.4f}, max={df_train[col].max():.4f}, mean={df_train[col].mean():.4f}")

print(f"\n所有列数: {len(df_train.columns)}")


使用设备: cpu
计算每个数据点到各类型设施最近的5个距离...

处理 hawker (sg-gov-hawkers.csv):
  原始数据点数: 107
  去重后数据点数: 106 (去除了 1 个重复位置)
  计算训练集最近的5个距离...
  计算测试集最近的5个距离...
  已创建 5 个特征列

处理 station (sg-mrt-stations.csv):
  原始数据点数: 243
  去重后数据点数: 192 (去除了 51 个重复位置)
  计算训练集最近的5个距离...
  计算测试集最近的5个距离...
  已创建 5 个特征列

处理 primaryschool (sg-primary-schools.csv):
  原始数据点数: 182
  去重后数据点数: 180 (去除了 2 个重复位置)
  计算训练集最近的5个距离...
  计算测试集最近的5个距离...
  已创建 5 个特征列

处理 secondaryschool (sg-secondary-schools.csv):
  原始数据点数: 153
  去重后数据点数: 152 (去除了 1 个重复位置)
  计算训练集最近的5个距离...
  计算测试集最近的5个距离...
  已创建 5 个特征列

处理 shoppingmall (sg-shopping-malls.csv):
  原始数据点数: 89
  去重后数据点数: 88 (去除了 1 个重复位置)
  计算训练集最近的5个距离...
  计算测试集最近的5个距离...
  已创建 5 个特征列

统一标准化所有最近距离特征（仅使用训练集计算标准化参数）...
训练集距离范围: [0.0032, 10.4481] 公里
训练集距离值数量: 4067275
注意: 使用训练集的min/max来标准化训练集和测试集，避免数据泄露

标准化所有最近距离特征...

保存文件...

文件已更新：train.csv 和 test.csv
训练集形状: (162691, 40)
测试集形状: (50000, 45)

创建的最近距离特征数: 25 个

特征列名示例（前10个）:
   1. DIST_NEAREST_5_HAWKER_1: min=0.3320, max=0.9969, mean=0.8

In [30]:
# 计算在指定距离范围内（1km, 3km, 5km, 10km, 20km）的各类型设施数量
# 对于每种设施类型，统计每个数据点在指定距离阈值内的设施数量

import pandas as pd
import numpy as np
import os
import torch

# 检查CUDA是否可用
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用设备: {device}")

def calculate_facility_counts_within_radius(df_main, df_aux, distance_thresholds, 
                                            lat_col='LATITUDE', lon_col='LONGITUDE', device=device):
    """
    计算主数据集中每个点在指定距离阈值内的辅助数据点数量（完全向量化版本）
    
    参数:
        df_main: 主数据集（训练集或测试集）
        df_aux: 辅助数据集（设施数据）
        distance_thresholds: 距离阈值列表（单位：公里），例如 [1, 3, 5, 10, 20]
        lat_col: 纬度列名
        lon_col: 经度列名
        device: 计算设备（'cuda' 或 'cpu'）
    
    返回:
        numpy数组，形状为 (len(df_main), len(distance_thresholds))，
        每行包含在每个距离阈值内的设施数量
    """
    # 提取主数据集的经纬度并转换为tensor
    main_lat = torch.tensor(df_main[lat_col].values, dtype=torch.float32, device=device)
    main_lon = torch.tensor(df_main[lon_col].values, dtype=torch.float32, device=device)
    
    # 提取辅助数据集的经纬度并转换为tensor
    aux_lat = torch.tensor(df_aux[lat_col].values, dtype=torch.float32, device=device)
    aux_lon = torch.tensor(df_aux[lon_col].values, dtype=torch.float32, device=device)
    
    # 创建掩码，标记有效的经纬度
    main_valid = ~(torch.isnan(main_lat) | torch.isnan(main_lon))
    aux_valid = ~(torch.isnan(aux_lat) | torch.isnan(aux_lon))
    
    # 过滤掉无效的辅助数据点
    aux_lat_valid = aux_lat[aux_valid]
    aux_lon_valid = aux_lon[aux_valid]
    
    if len(aux_lat_valid) == 0:
        return np.zeros((len(df_main), len(distance_thresholds)))
    
    # 转换为弧度
    main_lat_rad = torch.deg2rad(main_lat)
    main_lon_rad = torch.deg2rad(main_lon)
    aux_lat_rad = torch.deg2rad(aux_lat_valid)
    aux_lon_rad = torch.deg2rad(aux_lon_valid)
    
    # 使用广播计算所有距离矩阵
    # main_lat_rad: (N,), aux_lat_rad: (M,)
    # 扩展维度以进行广播: (N, 1) 和 (1, M) -> (N, M)
    main_lat_expanded = main_lat_rad.unsqueeze(1)  # (N, 1)
    main_lon_expanded = main_lon_rad.unsqueeze(1)  # (N, 1)
    aux_lat_expanded = aux_lat_rad.unsqueeze(0)    # (1, M)
    aux_lon_expanded = aux_lon_rad.unsqueeze(0)    # (1, M)
    
    # 计算Haversine距离（向量化）
    dlat = aux_lat_expanded - main_lat_expanded  # (N, M)
    dlon = aux_lon_expanded - main_lon_expanded  # (N, M)
    
    a = torch.sin(dlat/2)**2 + torch.cos(main_lat_expanded) * torch.cos(aux_lat_expanded) * torch.sin(dlon/2)**2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1-a))
    R = 6371.0  # 地球半径（公里）
    distances_matrix = R * c  # (N, M)
    
    # 将无效主数据点的距离设为inf，这样它们不会被计入
    distances_matrix[~main_valid, :] = float('inf')
    
    # 转换为numpy数组以便使用numpy的计数函数
    if device == 'cuda':
        distances_matrix = distances_matrix.cpu().numpy()
    else:
        distances_matrix = distances_matrix.numpy()
    
    # 对于每个距离阈值，统计在范围内的设施数量
    counts = np.zeros((len(df_main), len(distance_thresholds)))
    for i, threshold in enumerate(distance_thresholds):
        # 统计每个主数据点在阈值内的设施数量
        within_threshold = distances_matrix <= threshold
        counts[:, i] = within_threshold.sum(axis=1)
    
    # 对于无效的主数据点，将计数设为0
    if device == 'cuda':
        main_valid_np = main_valid.cpu().numpy()
    else:
        main_valid_np = main_valid.numpy()
    counts[~main_valid_np, :] = 0
    
    return counts

# 读取训练集和测试集
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# 定义设施类型和对应的文件名映射
facility_mapping = {
    'hawker': 'sg-gov-hawkers.csv',
    'station': 'sg-mrt-stations.csv',
    'primaryschool': 'sg-primary-schools.csv',
    'secondaryschool': 'sg-secondary-schools.csv',
    'shoppingmall': 'sg-shopping-malls.csv'
}

# 定义距离阈值（单位：公里）
distance_thresholds = [0.5, 1, 2, 3, 5, 10]

auxiliary_dir = "auxiliary-data"

print("="*60)
print(f"计算在指定距离范围内（{distance_thresholds}km）的各类型设施数量...")
print("="*60)

# 为每种设施类型计算在指定距离范围内的数量
for facility_type, filename in facility_mapping.items():
    aux_file = os.path.join(auxiliary_dir, filename)
    
    if not os.path.exists(aux_file):
        print(f"\n警告: 文件 {filename} 不存在，跳过 {facility_type}")
        continue
    
    # 读取辅助数据
    df_aux = pd.read_csv(aux_file)
    
    # 检查是否有LATITUDE和LONGITUDE列
    if 'LATITUDE' not in df_aux.columns or 'LONGITUDE' not in df_aux.columns:
        print(f"\n警告: {filename} 缺少LATITUDE或LONGITUDE列，跳过 {facility_type}")
        continue
    
    # 过滤掉缺失经纬度的行
    df_aux = df_aux.dropna(subset=['LATITUDE', 'LONGITUDE']).reset_index(drop=True)
    
    if len(df_aux) == 0:
        print(f"\n警告: {filename} 没有有效的经纬度数据，跳过 {facility_type}")
        continue
    
    # 对重复的经纬度进行去重（同一个位置只保留一个）
    # 这对于MRT站很重要，因为同一个站可能有多条线路
    original_count = len(df_aux)
    df_aux = df_aux.drop_duplicates(subset=['LATITUDE', 'LONGITUDE']).reset_index(drop=True)
    unique_count = len(df_aux)
    
    print(f"\n处理 {facility_type} ({filename}):")
    print(f"  原始数据点数: {original_count}")
    print(f"  去重后数据点数: {unique_count} (去除了 {original_count - unique_count} 个重复位置)")
    
    # 计算训练集在指定距离范围内的设施数量
    print(f"  计算训练集在 {distance_thresholds}km 范围内的设施数量...")
    train_counts = calculate_facility_counts_within_radius(
        df_train, df_aux, distance_thresholds, device=device
    )
    
    # 计算测试集在指定距离范围内的设施数量
    print(f"  计算测试集在 {distance_thresholds}km 范围内的设施数量...")
    test_counts = calculate_facility_counts_within_radius(
        df_test, df_aux, distance_thresholds, device=device
    )
    
    # 创建特征列并添加到DataFrame
    for i, threshold in enumerate(distance_thresholds):
        col_name = f"COUNT_WITHIN_{threshold}KM_{facility_type.upper()}"
        df_train[col_name] = train_counts[:, i]
        df_test[col_name] = test_counts[:, i]
    
    print(f"  已创建 {len(distance_thresholds)} 个特征列")
    
    # 显示统计信息
    print(f"  训练集统计（前3个阈值）:")
    for i, threshold in enumerate(distance_thresholds[:3]):
        col_name = f"COUNT_WITHIN_{threshold}KM_{facility_type.upper()}"
        mean_count = df_train[col_name].mean()
        max_count = df_train[col_name].max()
        print(f"    {threshold}km: 平均={mean_count:.2f}, 最大={max_count:.0f}")

# 保存修改后的文件
print("\n" + "="*60)
print("保存文件...")
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

print("\n" + "="*60)
print("文件已更新：train.csv 和 test.csv")
print(f"训练集形状: {df_train.shape}")
print(f"测试集形状: {df_test.shape}")

# 显示创建的特征统计
print(f"\n创建的距离范围内设施数量特征数: {len(facility_mapping) * len(distance_thresholds)} 个")
print("\n特征列名示例（前10个）:")
count_cols = [col for col in df_train.columns if col.startswith('COUNT_WITHIN_')]
for i, col in enumerate(count_cols[:10], 1):
    if col in df_train.columns:
        print(f"  {i:2d}. {col}: min={df_train[col].min():.0f}, max={df_train[col].max():.0f}, mean={df_train[col].mean():.2f}")

print(f"\n所有列数: {len(df_train.columns)}")


使用设备: cpu
计算在指定距离范围内（[0.5, 1, 2, 3, 5, 10]km）的各类型设施数量...

处理 hawker (sg-gov-hawkers.csv):
  原始数据点数: 107
  去重后数据点数: 106 (去除了 1 个重复位置)
  计算训练集在 [0.5, 1, 2, 3, 5, 10]km 范围内的设施数量...
  计算测试集在 [0.5, 1, 2, 3, 5, 10]km 范围内的设施数量...
  已创建 6 个特征列
  训练集统计（前3个阈值）:
    0.5km: 平均=0.40, 最大=5
    1km: 平均=1.22, 最大=8
    2km: 平均=3.25, 最大=18

处理 station (sg-mrt-stations.csv):
  原始数据点数: 243
  去重后数据点数: 192 (去除了 51 个重复位置)
  计算训练集在 [0.5, 1, 2, 3, 5, 10]km 范围内的设施数量...
  计算测试集在 [0.5, 1, 2, 3, 5, 10]km 范围内的设施数量...
  已创建 6 个特征列
  训练集统计（前3个阈值）:
    0.5km: 平均=0.43, 最大=5
    1km: 平均=1.46, 最大=9
    2km: 平均=5.07, 最大=25

处理 primaryschool (sg-primary-schools.csv):
  原始数据点数: 182
  去重后数据点数: 180 (去除了 2 个重复位置)
  计算训练集在 [0.5, 1, 2, 3, 5, 10]km 范围内的设施数量...
  计算测试集在 [0.5, 1, 2, 3, 5, 10]km 范围内的设施数量...
  已创建 6 个特征列
  训练集统计（前3个阈值）:
    0.5km: 平均=1.02, 最大=3
    1km: 平均=3.15, 最大=8
    2km: 平均=8.68, 最大=19

处理 secondaryschool (sg-secondary-schools.csv):
  原始数据点数: 153
  去重后数据点数: 152 (去除了 1 个重复位置)
  计算训练集在 [0.5, 1, 2, 3, 5, 10]km 范围内的

In [20]:
# ==========================================================
# CV + Hyperparam Search with tqdm, export CSV submissions
# Models: RandomForest / XGBoost / LightGBM
# ==========================================================
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from lightgbm import LGBMRegressor


TARGET_COL = "RESALE_PRICE"
features = [c for c in df_train.columns if c != TARGET_COL]
X = df_train[features]
y = df_train[TARGET_COL]
X_test = df_test[features]


def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def cv_search(model_name, model_ctor, param_grid, X, y, n_splits=5, random_state=42):
    best_params, best_rmse = None, float("inf")
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    grid = list(ParameterGrid(param_grid))
    pbar = tqdm(grid, desc=f"{model_name} grid", leave=True)

    for params in pbar:
        fold_rmses = []
        for tr_idx, va_idx in kf.split(X):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

            model = model_ctor(**params)
            model.fit(X_tr, y_tr)
            pred = model.predict(X_va)
            fold_rmses.append(rmse(y_va, pred))

        mean_rmse = float(np.mean(fold_rmses))
        if mean_rmse < best_rmse:
            best_rmse, best_params = mean_rmse, params
        pbar.set_postfix({"cv_rmse": f"{mean_rmse:.2f}", "best": f"{best_rmse:.2f}"})

    print(f"{model_name} best params: {best_params}")
    print(f"{model_name} best CV RMSE: {best_rmse:.2f}")
    return best_params, best_rmse

def train_full_and_predict(model_ctor, params, X, y, X_test):
    model = model_ctor(**params)
    model.fit(X, y)
    return model.predict(X_test)

def save_submission(preds, path):
    sub = pd.DataFrame({"Id": np.arange(len(preds)), "Predicted": preds})
    sub.to_csv(path, index=False)
    return sub


# rf_grid = {
#     "n_estimators": [300, 500],
#     "max_depth": [12, 20, 30],
#     "min_samples_split": [2, 5],
#     "min_samples_leaf": [1, 2],
#     "random_state": [42],
#     "n_jobs": [-1],
# }

lgbm_grid = {
    "n_estimators": [400, 700],
    "learning_rate": [0.05, 0.1],
    "max_depth": [-1, 8],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "reg_lambda": [0.0, 1.0],
    "random_state": [42],
    "n_jobs": [-1],
}


# rf_best_params, rf_cv = cv_search("RandomForest", RandomForestRegressor, rf_grid, X, y)

lgbm_best_params, lgbm_cv = cv_search("LightGBM", LGBMRegressor, lgbm_grid, X, y)


# rf_preds   = train_full_and_predict(RandomForestRegressor, rf_best_params, X, y, X_test)

lgbm_preds = train_full_and_predict(LGBMRegressor, lgbm_best_params, X, y, X_test)


# save_submission(rf_preds,   "rf_submission.csv")

save_submission(lgbm_preds, "lgbm_submission.csv")


cv_scores = {
    # "RandomForest": rf_cv,
    
    "LightGBM": lgbm_cv
}
best_name = min(cv_scores, key=cv_scores.get)
print(f"\n Best by CV RMSE: {best_name} ({cv_scores[best_name]:.2f})")

best_preds = {"LightGBM": lgbm_preds}[best_name]
save_submission(best_preds, "submission_best.csv")

print("\nFiles saved:")
# print(" - rf_submission.csv")
print(" - lgbm_submission.csv")
print(" - submission_best.csv (best by CV)")



LightGBM grid:   0%|          | 0/16 [00:00<?, ?it/s]


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: TOWN: object, BLOCK: object, STREET: object